In [4]:
#インポート
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
% matplotlib inline
import cv2

In [5]:
#ファイルの読み込み
train = pd.read_csv('leaf_train.csv')
#trainの大きさの確認
train.shape #(990, 194) 
#species
label = train['species'].values
label.shape

(990,)

In [6]:
#画像の読み込みを行う。
#初期化したlist型を作る
im = []
#1584枚の画像がある。
for i in range(1,1585):
    a = cv2.imread("images/%d.jpg" % i)
    gray = cv2.cvtColor(a, cv2.COLOR_BGR2GRAY)
    im.append(gray)


In [7]:
end_size = 100
a = np.max(im[1].shape)
height, width = im[1].shape
s = end_size/a
t = (int(width*s), int(height*s))
u = cv2.resize(im[1], t, cv2.COLOR_BGR2GRAY)
h, w = u.shape
print(height, h)
print(width, w)
v = int((w-h)/2)
print(v)
aa = np.zeros((v, 100))
bb = np.vstack((aa, u, aa))
bb.shape

251 37
667 100
31


(99, 100)

In [8]:
#サイズを統一する関数を定義
#比を変えないようにしながら、最終的に100×100になるようにする。
def a(a):
    import cv2
    #最後に持っていきたい大きさを定義
    end_size = 100
    #縦横で大きいほうの値を求める。
    f = max(im[a].shape) 
    #scaleに比を入れる。
    scale = end_size/f
    height, width = im[a].shape
    b = (int(width*scale), int(height*scale))
    c = cv2.resize(im[a], b, interpolation=cv2.INTER_CUBIC)
    height, width = c.shape
    if width > height:
        z_pad = np.zeros([int((100 - height)/2), width])
        d = np.vstack((z_pad, c, z_pad))
        e = cv2.resize(d, (100, 100))
        return e
    elif height > width: 
        z_pad = np.zeros([height, int((100 - width)/2)])
        d = np.hstack((z_pad, c, z_pad))
        e = cv2.resize(d, (100, 100))
        return e


In [9]:
for i in range(0,1584):
    cv2.imwrite("leaf_img/%d.jpg" % (i+1), a(i))

In [10]:
import numpy as np
import pandas as pd
import cv2
train = pd.read_csv('leaf_train.csv')
labels = train['species'].values
type(labels)
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(labels)
labels = le.transform(labels)
labels = labels.astype(np.int32)
rz_image = []
for i in train.id:
    image = cv2.imread('leaf_img/%d.jpg' % i)
    rz_image.append(image)
rz_image = np.array([rz_image])
rz_image = rz_image.reshape(-1, len(rz_image)).astype(np.float32)
rz_image /= 255
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(rz_image, label, test_size = 0.3, random_state = 0)
N_train = y_train.size
N_test = y_test.size

In [11]:
import chainer
import chainer.functions as F
import chainer.links as L
from chainer import computational_graph
from chainer import serializers

In [12]:
class MLP(chainer.Chain):
    def __init__(self):
        super(MLP, self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(10000, 1000)
            self.l2 = L.Linear(1000, 500)
            self.l3 = L.Linear(500, 700)
            self.l4 = L.Linear(700, 99)
    
    def __call__(self, X):
        h1 = F.relu(self.l1(X))
        h2 = F.relu(self.l2(h1))
        h3 = F.relu(self.l3(h2))
        return self.l4(h3)

In [20]:
gpu = -1

In [21]:
if gpu >= 0:
    xp = cuda.cupy
    cupy.cuda.Device(gpu).use()
    model.to_gpu()
else:
    xp = np

In [22]:
model = L.Classifier(MLP())
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)
batchsize = 100
n_epoch = 20

In [23]:
from chainer.datasets import tuple_dataset
from chainer import iterators, training
from chainer.training import extensions

In [24]:
tr = tuple_dataset.TupleDataset(X_train, y_train)
tr_iter = iterators.SerialIterator(tr, batch_size = batchsize, shuffle = True)
updater = training.StandardUpdater(tr_iter, optimizer, device = gpu)
trainer = training.Trainer(updater, (n_epoch, "epoch"), out = 'result')

In [25]:
test = tuple_dataset.TupleDataset(X_test, y_test)
test_iter = iterators.SerialIterator(test, batch_size = batchsize, shuffle = False, repeat = False)
trainer.extend(extensions.Evaluator(test_iter, model))
trainer.extend(extensions.LogReport())
trainer.extend(extensions.PrintReport(['epoch', 'main/loss', 'main/accuracy', 'validation/main/loss','validation/main/accuracy']))
trainer.extend(extensions.PlotReport(['main/accuracy', 'validation/main/accuracy'],
                          'epoch', file_name='accuracy.png'))
# trainとtestの損失関数のグラフを出力
trainer.extend(extensions.PlotReport(['main/loss', 'validation/main/loss'],
                          'epoch', file_name='leaf_classifier_loss.png'))
trainer.extend(extensions.ProgressBar())

In [26]:
trainer.run()

TypeError: string indices must be integers